## Load Metadata - System Tables
- Prepare metadata and merge into the control table
- Clone notebook for each source schema

In [0]:
-- Create temporary view containing table specific metadata
-- This will be joined with data from information_schema

create or replace temp view table_details as
select
  col1 as table,
  col2 as sink_table,
  split(col3, ',') as primary_key,
  split(col4, ',') as cluster_cols,
  col5 as load_type,
  col6 as watermark_col_name,
  col7 as watermark_col_type,
  col8 as watermark_col_start_value
FROM
  (
    VALUES
      ("history",    "query_history",      "account_id,workspace_id,statement_id", "start_time,workspace_id",    "incremental", "update_time", "timestamp", "0000-01-01"),
      ("warehouses", "compute_warehouses", null,                                   null,                         "full",        null,          null,        null)
  );

In [0]:
use catalog identifier(:ctrl_catalog);
use schema identifier(:ctrl_schema);

create or replace temporary view control_src as
select
  'lakefed_ingest' as job_name,
  'system_tables' as task_collection,
  'delta' as src_type,
  t.table_catalog as src_catalog,
  t.table_schema as src_schema,
  t.table_name as src_table,
  'lakefed_ingest' as sink_catalog,
  'system_tables' as sink_schema,
  coalesce(d.sink_table, t.table_name) as sink_table,
  coalesce(d.primary_key, array()) as primary_key,
  coalesce(d.cluster_cols, array()) as sink_cluster_cols,
  coalesce(d.load_type, 'full') as load_type,
  false as load_partitioned,
  '*' as select_list,
  coalesce(d.watermark_col_name, null) as watermark_col_name,
  coalesce(d.watermark_col_type, null) as watermark_col_type,
  coalesce(d.watermark_col_start_value, null) as watermark_col_start_value,
  case when load_partitioned is true then element_at(cluster_cols, 1) else null end as partition_col,
  case when load_partitioned is true then 256 else null end as partition_size_mb,
  true as task_enabled
from system.information_schema.tables t
left join table_details d
on t.table_name = d.table
where t.table_catalog = 'system'
  and t.table_schema in ('query', 'compute')
  and t.table_name in (select table from table_details);

select * from control_src;

In [0]:
-- This will overwrite existing watermarks if they exist
-- If this is not desired, exclude existing rows from the source

merge with schema evolution into identifier(:ctrl_table) as t
using control_src as s
on t.src_catalog = s.src_catalog
and t.src_schema = s.src_schema
and t.src_table = s.src_table
when matched
  then update set *
when not matched
  then insert *